# TODO dorobić ile połączeń występuje

In [1]:
from polish_parser import Parser

my_parser = Parser()

In [8]:
my_parser.parse("zamki ma ")

Position: 6
Length: 6
Expecting: [słuchaliby, słuchali, słuchają]
Reason: Verb should match the noun number: plural. But is singular.

In [6]:
import tkinter as tk

class GrammarEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Grammar Learning Editor")
        self.text = tk.Text(root, wrap="word", undo=True)
        self.text.pack(fill="both", expand=True)
        self.text.focus_set()

        self.text.bind("<KeyRelease>", self.on_text_change)
        self.text.bind("<Tab>", self.on_tab)
        self.root.bind_all("<Up>", self.on_up, add="+")
        self.root.bind_all("<Down>", self.on_down, add="+")
        self.text.tag_configure("error", underline=True, foreground="red")

        # Popup for suggestions with scrollbar
        popup_frame = tk.Frame(root)
        
        self.popup_scrollbar = tk.Scrollbar(popup_frame, orient="vertical")
        self.popup = tk.Listbox(popup_frame, height=10, yscrollcommand=self.popup_scrollbar.set)
        self.popup_scrollbar.config(command=self.popup.yview)
        
        # Pack inside the frame (Listbox on left, scrollbar on right)
        self.popup.pack(side="left", fill="both", expand=True)
        self.popup_scrollbar.pack(side="right", fill="y")
        
        # Store frame instead of Listbox directly for placement
        self.popup_frame = popup_frame
        
        self.popup.bind("<Double-Button-1>", self.apply_suggestion)
        self.popup.bind("<Return>", self.apply_suggestion)
        self.popup_is_visible = False

        self.selected: int = 0
        self.suggestions = []
        self.error_pos = None

    def on_text_change(self, event=None):
        text = self.text.get("1.0", "end-1c")
        
        current_word = "" if text == "" or text.endswith(" ") else text.split()[-1]
        result = my_parser.parse(text)

        # Clear error tag
        self.text.tag_remove("error", "1.0", "end")

        # Hide popup if not needed
        self.suggestions = list()
        if result is None:
            self.hide_popup()
            return
    
        self.suggestions = [w.word for w in result.expected]
        
        self.error_pos = result.position

        # Get text index for red underline
        index = self.text.index(f"1.0+{result.position}c")
        next_space = self.text.search(r"\s", index, regexp=True)
        if not next_space:
            next_space = "end"

        self.text.tag_add("error", index, next_space)
        self.show_popup(index, self.suggestions)

    def show_popup(self, index, suggestions):
        bbox = self.text.bbox(index)
        if not bbox:
            self.hide_popup()
            return
    
        x, y, width, height = bbox
        self.popup_frame.place(x=x, y=y + height)
        self.popup.delete(0, tk.END)
        for s in suggestions:
            self.popup.insert(tk.END, s)
        self.popup_is_visible = True
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(self.selected)
        self.popup.see(self.popup.curselection())
    
    def hide_popup(self):
        self.popup_frame.place_forget()
        self.popup_is_visible = False


    def on_tab(self, event):
        if self.popup_is_visible and self.suggestions:
            suggestion = self.suggestions[self.popup.curselection()[0]]
            self.insert_suggestion(suggestion)
            return "break"  # Prevent default tab
        return None
    
    def on_up(self, event):
        """Navigate popup list upward."""
        if not self.popup_is_visible:
            return None
        cur = self.popup.curselection()
        if not cur:
            new = 0
        else:
            new = (cur[0] - 1) % self.popup.size()
        self.selected = new
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(new)
        self.popup.activate(new)
        self.popup.see(new)  # 👈 ensure visible in scroll region
        return "break"
    
    def on_down(self, event):
        """Navigate popup list downward."""
        if not self.popup_is_visible:
            return None
        cur = self.popup.curselection()
        if not cur:
            new = 0
        else:
            new = (cur[0] + 1) % self.popup.size()
        self.selected = new
        self.popup.selection_clear(0, tk.END)
        self.popup.selection_set(new)
        self.popup.activate(new)
        self.popup.see(new)  # 👈 ensure visible in scroll region
        return "break"

    def apply_suggestion(self, event):
        if self.popup_is_visible:
            suggestion = self.suggestions[self.popup.curselection()[0]]
            self.insert_suggestion(suggestion)

    def insert_suggestion(self, suggestion):
        if self.error_pos is not None:
            start = self.text.index(f"1.0+{self.error_pos}c")
            next_space = self.text.search(r"\s", start, regexp=True)
            if not next_space:
                next_space = "end"
            print(start)
            self.text.delete(start, next_space)
            self.text.insert(start, suggestion + " ")
            self.hide_popup()
            self.on_text_change()


if __name__ == "__main__":
    root = tk.Tk()
    app = GrammarEditor(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\mateu\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\mateu\AppData\Local\Temp\ipykernel_10904\3310494609.py", line 65, in on_text_change
    self.show_popup(index, self.suggestions)
  File "C:\Users\mateu\AppData\Local\Temp\ipykernel_10904\3310494609.py", line 81, in show_popup
    self.popup.see(self.popup.curselection())
  File "C:\Users\mateu\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 3318, in see
    self.tk.call(self._w, 'see', index)
_tkinter.TclError: bad listbox index "": must be active, anchor, end, @x,y, or a number


1.0
1.6
1.13
